# IP Score

Currently, we transform each IP into bits and traverse the binary tree summing all the encountered counters (it forms `score(IP)`). The final similarity score for an IP, is given by `score(IP)/max_possible_score`. 

The problem is that we calculate `max_possible_score` using `32*root_counter`. This is the maximum possible score for a tree with `root_counter` same IPs inside, and not the particular tree in question. For a balance tree, the maximum possible score will be much lower. This is problematic, as for more or less balanced tree, we'll never achieve similarity scores close to 1. Even if the node is being spammed by an attacker. 

The below shows what's the IP score when there's an attacker creates the same amount of nodes as all the honest nodes but using a single IP. 

In [ ]:
#!/usr/bin/python3 

import math
import matplotlib.pyplot as plt

from Tree import *
from TreeOnur import *

# helper function to flip a bit written as a string/character
def flip(b):
    assert(b == '0' or b == '1')
    if (b == '0'):
        return '1'
    return '0'

# generates a specified amount of IPs that will get the lowest possible score in the IP tree
def generate_IPs(n):
    ips = []
    init_ip = list('1'*32)
    for i in range(0, n):
        for j in range(0, len(init_ip)):
            if((i % (2**j)) == 0):
                init_ip[j] = flip(init_ip[j])
        
        ip_str = ''
        for octet in range(0, 4):
            offset = octet*8
            octet_str = str(int(''.join(init_ip[offset:offset + 8]), 2))
            ip_str  = ip_str + '.' + octet_str
        #remove the first '.'
        ips.append(ip_str[1:])
    return ips

ips = generate_IPs(2000)
#print(ips)
print("There are",len(ips), "ips and", len(set(ips)), "distinct ips.")

tree = Tree()
treeOnur = TreeOnur()
scores = []
fig = plt.figure()
ax = fig.add_subplot()
malicious_ips_num = 1

counter = 0
for ip in ips:
    score = tree.add(ip)
    scoreOnur = treeOnur.add(ip)
    #print("added:", ip, "with score:", score)
    #print("######################################################")
    ax.scatter(2*counter, score, label='benign', color='green')
    ax.scatter(2*counter, scoreOnur, label='benign', color='blue', marker='x')
    
    score = tree.add(ips[counter % malicious_ips_num])
    scoreOnur = treeOnur.add(ips[counter % malicious_ips_num])
    ax.scatter(2*counter + 1, score, label='malicious', color='red')
    ax.scatter(2*counter + 1, scoreOnur, label='malicious', color='orange', marker='x')
    counter += 1

plt.show()

We make it better by taking the IP similarity score to the power of `P_ip`, which helps a bit, but it seems more like a hack rather than a solution. Specifically, solving the issue with a specific `P_ip` might backfire against honest nodes in some cases. 

The issue is getting worse when we give the attacker 10 different IPs. 

In [ ]:
tree = Tree()
treeOnur = TreeOnur()
scores = []
fig = plt.figure()
ax = fig.add_subplot()
malicious_ips_num = 20

counter = 0
for ip in ips:
    score = tree.add(ip)
    scoreOnur = treeOnur.add(ip)
    #print("added:", ip, "with score:", score)
    #print("######################################################")
    ax.scatter(2*counter, score, label='benign', color='green')
    ax.scatter(2*counter, scoreOnur, label='benign', color='blue', marker='x')
    
    score = tree.add(ips[counter % malicious_ips_num])
    scoreOnur = treeOnur.add(ips[counter % malicious_ips_num])
    ax.scatter(2*counter + 1, score, label='malicious', color='red')
    ax.scatter(2*counter + 1, scoreOnur, label='malicious', color='orange', marker='x')
    counter += 1

plt.show()

The question is "what to do about it?". We can change the `max_score` so it's the maximum value possible for this specific tree. However, if we do that, a normal IP in a perfectly balanced tree would yield high ip similarity score which IMO we should avoid. 

A more robust approach might be to use some measure of how the tree is balanced and whether the new IP is increasing or decreasing this balance metric (and by how much).

One idea for the "balance score" would be to take the difference between the max and min possible score and normalize them with the `root_counter`. To investigate. 

In [ ]:
from Tree import *
from TreeOnur import *
import math
import matplotlib.pyplot as plt

tree = Tree()
#treeOnur = TreeOnur()
scores = []
fig = plt.figure()
ax = fig.add_subplot()
malicious_ips_num = 10

# helper function to flip a bit written as a string/character
def flip(b):
    assert(b == '0' or b == '1')
    if (b == '0'):
        return '1'
    return '0'

# generates a specified amount of IPs that will get the lowest possible score in the IP tree
def generate_IPs(n):
    ips = []
    init_ip = list('1'*32)
    for i in range(0, n):
        for j in range(0, len(init_ip)):
            if((i % (2**j)) == 0):
                init_ip[j] = flip(init_ip[j])
        
        ip_str = ''
        for octet in range(0, 4):
            offset = octet*8
            octet_str = str(int(''.join(init_ip[offset:offset + 8]), 2))
            ip_str  = ip_str + '.' + octet_str
        #remove the first '.'
        ips.append(ip_str[1:])
    return ips

ips = generate_IPs(600)

counter = 0
for i in range(0, 10):
    ip = ips[counter % malicious_ips_num]
    score = tree.add(ip)
    #scoreOnur = treeOnur.add(ip)
    print("added:", ip, "with score:", score)
    ax.scatter(i, score, label='malicious', color='red')
    #ax.scatter(i, scoreOnur, label='malicious', color='orange', marker='x')
    counter += 1

plt.show()